# Fluidize-Python Interactive Demo

This notebook demonstrates the fluidize-python library for managing scientific computing projects.

## Setup

First, let's import the client and see where our projects will be stored:

In [1]:
# Import the fluidize client - handlers auto-register!
from fluidize.client import FluidizeClient

# Create client and config
client = FluidizeClient(mode="local")

print(f"📁 Projects will be stored in: {client.config.local_projects_path}")
print(f"📁 Base directory: {client.config.local_base_path}")
print(f"🚀 Client ready in '{client.mode}' mode!")

📁 Projects will be stored in: /Users/henrybae/.fluidize/projects
📁 Base directory: /Users/henrybae/.fluidize
🚀 Client ready in 'local' mode!


## 1. Creating Projects

Let's create some projects with different configurations:

In [19]:
# Create a comprehensive project
project1 = client.projects.create(
    project_id="MUJOCO",
    label="MUJOCO DEMO",
    description="A MuJoCo simulation project",
    status="active",
)

print("✅ Created project 1:")
print(f"   ID: {project1.id}")
print(f"   Label: {project1.label}")
print(f"   Status: {project1.status}")

✅ Created project 1:
   ID: MUJOCO
   Label: MUJOCO DEMO
   Status: active


In [20]:
# Get all projects
projects = client.projects.list()

print(f"📋 Found {len(projects)} projects:")
print()

for i, project in enumerate(projects, 1):
    print(f"{i:2}. {project.id}")
    print(f"     Label: {project.label}")
    print(f"     Status: {project.status}")
    if project.description:
        print(f"     Description: {project.description[:50]}{'...' if len(project.description) > 50 else ''}")
    print()

📋 Found 2 projects:

 1. data-pipeline-2024
     Label: Data Processing Pipeline
     Status: active
     Description: A comprehensive data processing pipeline for custo...

 2. MUJOCO
     Label: MUJOCO DEMO
     Status: active
     Description: A MuJoCo simulation project



## 2. Creating Nodes

In [7]:
# Import required types for creating nodes from scratch
import datetime

from fluidize.core.types.graph import GraphNode, Position, graphNodeData
from fluidize.core.types.node import nodeMetadata_simulation, nodeProperties_simulation

print("📦 Imported all node creation types successfully!")

📦 Imported all node creation types successfully!


### Creating Nodes in Two Ways

Fluidize supports two approaches for creating nodes in your project graph:

1. **`add_node()`** - For nodes that use existing simulation templates
2. **`add_node_from_scratch()`** - Complete node creation with all files and directories

Let's demonstrate the comprehensive `add_node_from_scratch` approach that creates:
- Graph node entry in `graph.json`
- Complete `properties.yaml` with container configuration
- Rich `metadata.yaml` with authors, tags, and references
- Source directory (optionally cloned from a repository)

In [15]:
# Get our MUJOCO project for node creation
project = client.projects.get("MUJOCO")

print(f"🎯 Working with project: {project.label}")
print(f"📊 Current graph state: {len(project.graph.get().nodes)} nodes, {len(project.graph.get().edges)} edges")

🎯 Working with project: MUJOCO DEMO
📊 Current graph state: 1 nodes, 0 edges


### Complete Node Creation Example

Let's create a comprehensive MuJoCo simulation node with all metadata:

In [12]:
# 1. Create the GraphNode (defines position and basic info in the graph)
mujoco_graph_node = GraphNode(
    id="Mujoco-Simulation",
    position=Position(x=150.0, y=100.0),
    data=graphNodeData(label="MuJoCo Humanoid Simulation"),
    type="physics-simulation",
)

print("✅ Created GraphNode:")
print(f"   ID: {mujoco_graph_node.id}")
print(f"   Label: {mujoco_graph_node.data.label}")
print(f"   Position: ({mujoco_graph_node.position.x}, {mujoco_graph_node.position.y})")

✅ Created GraphNode:
   ID: Mujoco-Simulation
   Label: MuJoCo Humanoid Simulation
   Position: (150.0, 100.0)


In [ ]:
# 2. Create nodeProperties (container and execution configuration)
mujoco_properties = nodeProperties_simulation(
    # Required fields
    container_image="",  # Docker image with CUDA support
    simulation_mount_path="source",  # Mount path inside container
    source_output_folder="source/outputs",  # Where outputs are stored
)

mujoco_metadata = nodeMetadata_simulation(
    name="MuJoCo Humanoid Locomotion Demo",
    id="mujoco-humanoid-demo",
    version="0.1",
    description="A demo simulation of a humanoid in MuJoCo",
    date=datetime.date.today(),
    authors=[],
    tags=[],
)

print("✅ Created nodeProperties:")
print(f"   Container: {mujoco_properties.container_image}")
print(f"   Mount path: {mujoco_properties.simulation_mount_path}")
print(f"   Output folder: {mujoco_properties.source_output_folder}")
print(f"   Should run: {mujoco_properties.should_run}")

✅ Created nodeProperties:
   Container: nvidia/cuda:11.8-devel-ubuntu20.04
   Mount path: source
   Output folder: source/outputs
   Should run: True


In [14]:
# 4. Create the complete node from scratch!
print("🚀 Creating node from scratch...")

created_node = project.graph.add_node_from_scratch(
    node=mujoco_graph_node,
    node_properties=mujoco_properties,
    node_metadata=mujoco_metadata,
)

print("✅ Node created successfully!")
print(f"   Node ID: {created_node.id}")
print(f"   Label: {created_node.data.label}")
print(f"   Type: {created_node.type}")
print(f"   Simulation ID: {created_node.data.simulation_id}")

🚀 Creating node from scratch...
Info: Optional field 'image_name' not provided in nodeProperties
Info: Optional field 'last_run' not provided in nodeProperties
Info: Optional field 'code_url' not provided in nodeMetadata
Info: Optional field 'paper_url' not provided in nodeMetadata
Info: Optional field 'tags' not provided in nodeMetadata
✅ Node created successfully!
   Node ID: Mujoco-Simulation
   Label: MuJoCo Humanoid Simulation
   Type: physics-simulation
   Simulation ID: None


## 3. Retrieving Specific Projects

Get detailed information about a specific project:

In [2]:
# Get project details
project = client.projects.get("MUJOCO")

print("📊 Project Details:")
print(f"   ID: {project.id}")
print(f"   Label: {project.label}")
print(f"   Description: {project.description}")
print(f"   Status: {project.status}")
print(f"   Location: {project.location}")
print(f"   Metadata Version: {project.metadata_version}")

📊 Project Details:
   ID: MUJOCO
   Label: MUJOCO DEMO
   Description: A MuJoCo simulation project
   Status: active
   Location: 
   Metadata Version: 1.0


In [ ]:
from fluidize.core.types.runs import RunFlowPayload

payload = RunFlowPayload(
    name="simulation-run-1", description="Running simulation flow", tags=["simulation", "analysis"]
)


project.runs.run_flow(payload)

No start node provided, using first node: Mujoco-Simulation
BFS traversal starting from node 'Mujoco-Simulation':
  - Adding node to traversal: Mujoco-Simulation, previous node: None
Nodes to run: ['Mujoco-Simulation']
Created project run folder: /Users/henrybae/.fluidize/projects/MUJOCO/runs/run_10
Created run environment with number: 10


{'flow_status': 'running', 'run_number': 10}

No parameters.json found for node Mujoco-Simulation


Executing node Mujoco-Simulation in run 10

=== Starting run for node: Mujoco-Simulation ===
1. Preparing environment...
🔍 [Environment] DEBUG: Attempting to load parameters from /Users/henrybae/.fluidize/projects/MUJOCO/runs/run_10/Mujoco-Simulation
🔍 [Environment] DEBUG: Loaded param_data = {'metadata': {'description': 'Parameter tuning for pinata motor strength', 'version': '1.0'}, 'parameters': [{'name': 'motor_strength', 'value': '20.0', 'type': 'text', 'label': 'Motor Strength', 'description': 'Control signal strength for bat motor (higher = faster swing, more collision force)', 'scope': 'simulation', 'location': ['source/pinata_simulation.py']}]}
🔍 [Environment] DEBUG: all_params = [{'name': 'motor_strength', 'value': '20.0', 'type': 'text', 'label': 'Motor Strength', 'description': 'Control signal strength for bat motor (higher = faster swing, more collision force)', 'scope': 'simulation', 'location': ['source/pinata_simulation.py']}]
🔍 [Environment] DEBUG: Found 1 simulation p

## 4. Updating Projects

Modify existing projects: